In [ ]:
import os
import numpy as np
from scipy.signal import find_peaks
from backtesting import Backtest, Strategy
from backtesting.lib import FractionalBacktest


# Data

In [68]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()

DB_URL = os.getenv('DB_URL')

engine = create_engine(
    DB_URL,
    pool_pre_ping=True,   # checks connection before using
    pool_recycle=1800,    # optional: avoids stale timeouts
    connect_args={"check_same_thread": False} if "sqlite" in DB_URL else {},
)
# engine

In [108]:
import pandas as pd
from datetime import datetime
SCHEMA = 'proddb.'
tables = {
    'p5m': SCHEMA+'coin_prices_5m',
    'p1h': SCHEMA+'coin_prices_1h',
    'f5m': SCHEMA+'f_coin_signal_5m',
    'f10m': SCHEMA+'f_coin_signal_10m',
    'f15m': SCHEMA+'f_coin_signal_15m',
    'f30m': SCHEMA+'f_coin_signal_30m',
    'f1h': SCHEMA+'f_coin_signal_1h',
    'f4h': SCHEMA+'f_coin_signal_4h',
    'f1d': SCHEMA+'f_coin_signal_1d',
    'f1D': SCHEMA+'f_coin_signal_1d',
    'orders': SCHEMA+'trade_orders_sim',
    'tp_by_sess': SCHEMA+'trade_orders_tp_by_session',
    }

from_time = int(datetime(2025, 4, 1, 0, 0, 0).timestamp())
to_time = int(datetime(2025, 9, 1, 0, 0, 0).timestamp())
from_time, to_time
def get_data_4_8(symbol, tf='f1h', extr_cols=[]):
    try:
        table = tables[tf]
    except Exception as e:
        print(e)
        return tables['p1h']
    
    if len(extr_cols) > 0:
        extr_cols_str = ', ' + ', '.join(extr_cols)
    else:
        extr_cols_str = ''

    df = pd.read_sql(f"""
            select TO_TIMESTAMP(open_time) as open_time, open as Open, close as Close,
                high  as High, low as Low, volume as Volume 
                {extr_cols_str}
            from {table} 
            where open_time >= {from_time} and open_time < {to_time}
                and symbol = '{symbol}'
            order by open_time asc
        """,
        engine, 
        index_col="open_time")
    df.rename(columns={
        'open': 'Open',
        'close': 'Close',
        'high': 'High',
        'low': 'Low',
        'volume': 'Volume',
        'rsi7': 'rsi'
    }, inplace=True)
    return df
df = get_data_4_8('HBARUSDT', 'f1h', ['rsi7', 'adx', 'tr', 'atr14'])

In [109]:
# time = df.index
# for i in range(1, len(time)):
#     if (time[i] - time[i-1]).total_seconds() == 3600:
#         print(i, time[i], time[i-1])
#         break

df 

,Open,Close,High,Low,Volume,rsi,adx,tr,atr14
open_time,,,,,,,,,
2025-05-13 05:00:00+00:00,0.20554,0.20539,0.20610,0.20419,5471093.0,0.000000,0.000000,0.00191,0.001910
2025-05-13 06:00:00+00:00,0.20539,0.20618,0.20773,0.20522,8284475.0,100.000000,1.714952,0.00251,0.001953
2025-05-13 07:00:00+00:00,0.20617,0.20677,0.20905,0.20599,6642622.0,100.000000,3.260341,0.00306,0.002032
2025-05-13 08:00:00+00:00,0.20678,0.20718,0.20833,0.20651,5241162.0,100.000000,4.445942,0.00182,0.002017
2025-05-13 09:00:00+00:00,0.20719,0.20992,0.21034,0.20712,9200234.0,100.000000,5.349335,0.00322,0.002103
...,...,...,...,...,...,...,...,...,...
2025-08-31 12:00:00+00:00,0.22497,0.22484,0.22513,0.22348,2047893.0,42.469685,3.935555,0.00165,0.001790
2025-08-31 13:00:00+00:00,0.22483,0.22345,0.22519,0.22318,2397163.0,32.524784,3.836092,0.00201,0.001806
2025-08-31 14:00:00+00:00,0.22345,0.22352,0.22400,0.22237,3075512.0,33.440501,4.004198,0.00163,0.001793


# Backtesting lib

In [ ]:

from backtesting.test import GOOG

GOOG.tail()
def save_stats(stats, backtest, out='backtest_summary.csv',tag=''):
    d = stats.to_dict()
    _ = d.pop('_equity_curve')
    trades = d.pop('_trades')
    df = pd.DataFrame([d])
    df['tag'] = tag
    df.to_csv(out, header=False, index=False, mode='a')

    folder = tag.replace('_', '/')
    os.makedirs(folder, exist_ok=True)
    trades.to_markdown(os.path.join(folder, str(stats._strategy)+'_trades.md'), index=False)
    backtest.plot(filename=os.path.join(folder, str(stats._strategy)+'_equity_curve.html'), open_browser=False)



## Triple pattern

In [101]:
def detect_triple_pattern(df, tol=0.01, min_prominence=0.01, min_distance=5):
    """Identify Triple Top or Bottom patterns in OHLC data.
    Parameters:
    - df: DataFrame with 'Close' prices
    - kind: 'top' for Triple Top or 'bottom' for Triple Bottom
    - tol: Tolerance for matching levels as fraction
    - min_prominence: Minimum prominence for swing detection
    - min_distance: Minimum candle distance between swings
    Returns:
    - List of dictionaries with indices of pattern points and necklines
    """
    df = df.copy()
    lows = df['Low']
    highs = df['High']
    swings, _ = find_peaks(-lows, prominence=min_prominence, distance=min_distance)
    # direction = 'bullish'
    bullish = []
    for i in range(len(swings) - 2):
        p1, p2, p3 = swings[i], swings[i+1], swings[i+2]
        levels = lows.values[[p1, p2, p3]]
        lv_mean = levels.mean()
        if np.ptp(levels) <= tol * lv_mean:
            neckline_val = max(highs.values[p1:p3+1])
            bullish.append({
                'p1': p1, 'p2': p2, 'p3': p3,
                'boundary_value': neckline_val,
                })
            

    swings, _ = find_peaks(highs, prominence=min_prominence, distance=min_distance)
    # direction = 'bearish'
    bearish = []
    for i in range(len(swings) - 2):
        p1, p2, p3 = swings[i], swings[i+1], swings[i+2]
        levels = highs.values[[p1, p2, p3]]
        lv_mean = levels.mean()
        if np.ptp(levels) <= tol * lv_mean:
            neckline_val = min(lows.values[p1:p3+1])
            bearish.append({
                'p1': p1, 'p2': p2, 'p3': p3,
                'boundary_value': neckline_val,
                })
    return (bullish, bearish)


In [102]:
class TriplePatternStrategy(Strategy):
    max_periods = 9
    tol = 0.01
    min_prominence = 0.01
    min_distance = 5
    def init(self,):
        df = self.data.df
        # print(self.max_periods)
        bull, bear = detect_triple_pattern(df, tol=self.tol, min_prominence=self.min_prominence, min_distance=self.min_distance)
        # print(bull, bear)
        df['bull'] = None
        
        self.bull = []
        self.bear = []

        for p in bull:
            df['bull'].iat[p['p3']] = p['boundary_value']
            self.bull.append([p['p3'], p['boundary_value']])

        df['bear'] = None
        for p in bear:
            df['bear'].iat[p['p3']] = p['boundary_value']
            self.bear.append([p['p3'], p['boundary_value']])

        # self.bull = self.I(lambda: df['bull'], name='Bullish')
        # self.bear = self.I(lambda: df['bear'], name='Bearish')

    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) -1
        for trade in self.trades:
            if current_bar - trade.entry_bar >=  periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)
    def next(self):
        self.CloseOldPosition(self.max_periods)
        if len(self.bull) > 0:
            t, boundary = self.bull[0]
            if len(self.data.index) > t+1:
                if self.data.High[-1] > boundary:
                    if not self.position.is_long:
                        # print("BULL: ", self.data.index[-1], t, boundary, self.data.Close[-1], self.data.High[-1], self.data.Low[-1])
                        self.buy(sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP
                        self.position.close()  # Close previous position if any
                    self.bull.pop(0)
                elif len(self.data.index) > t+14:
                    self.bull.pop(0)

        if len(self.bear) > 0:
            t, boundary = self.bear[0]
            if len(self.data.index) > t+1:
                if self.data.Low[-1] < boundary:
                    if not self.position.is_short:
                        # print("BEAR: ", self.data.index[-1], t, boundary, self.data.Close[-1], self.data.High[-1], self.data.Low[-1])
                        # self.sell(sl=self.data.Close[-1] * 1.02, tp=self.data.Close[-1] * 0.95)  # Example SL/TP
                        self.position.close()  # Close previous position if any
                    self.bear.pop(0)
                elif len(self.data.index) > t+14:
                    self.bear.pop(0)

class TriplePatternStrategy_9p(TriplePatternStrategy):
    max_periods = 9
    tol = 0.01
    min_prominence = 0.01
    min_distance = 5

class TriplePatternStrategy_26p(TriplePatternStrategy):
    max_periods = 26
    tol = 0.01
    min_prominence = 0.01
    min_distance = 5
 

In [16]:
# df = GOOG.copy()

# bt = FractionalBacktest(df, TriplePatternStrategy_26p, cash=10_000, commission=.002, fractional_unit=0.001)
# stats = bt.run()
# stats.to_csv("backtest_results.csv", )


In [ ]:
# bt.plot()

## Butterfly Pattern

In [103]:
from pyharmonics.marketdata import BinanceCandleData  # or your own DataFrame
from pyharmonics.technicals import Technicals
from pyharmonics.search import HarmonicSearch
import pandas as pd

def detect_butterfly_pattern(df):
    _df = df.copy()
    _df.columns = ['open', 'high', 'low', 'close', 'volume']
    t = Technicals(_df, 'SYMBOL', 'CUSTOM')  # timeframe as needed
    hs = HarmonicSearch(t)
    hs.search()
    patterns = hs.get_patterns(family=hs.XABCD)  # includes Butterfly 
    return patterns['XABCD']

In [104]:
class ButterflyStrategy(Strategy):
    max_periods = 9
    def init(self):
        df = self.data.df
        # print(self.max_periods)
        patterns = detect_butterfly_pattern(df)
        # print(patterns)
        df['bull'] = 0
        df['bear'] = 0
        if len(patterns) > 0:            
            for p in patterns:
                if p.bullish:
                    df['bull'].at[p.x[-1]] = 1
                else:
                    df['bear'].at[p.x[-1]] = 1

        self.bull = self.I(lambda: df['bull'], name='Bullish')
        self.bear = self.I(lambda: df['bear'], name='Bearish')

    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) -1
        for trade in self.trades:
            if current_bar - trade.entry_bar >= periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)
    def next(self):
        self.CloseOldPosition(self.max_periods)
        if self.bull > 0:
            if not self.position.is_long:
                self.buy(sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP
                self.position.close()  # Close previous position if any
        if self.bear > 0:
            if not self.position.is_short:
                # self.sell(sl=self.data.Close[-1] * 1.02, tp=self.data.Close[-1] * 0.95)  # Example SL/TP
                self.position.close()  # Close previous position if any

class ButterflyStrategy_9p(ButterflyStrategy):
    max_periods = 9

class ButterflyStrategy_26p(ButterflyStrategy):
    max_periods = 26
 

In [ ]:
# bt = FractionalBacktest(df, ButterflyStrategy_9p, cash=10_000, commission=.002, fractional_unit=0.01)
# stats = bt.run()
# bt.plot()


## RSI long Pattern

In [44]:
class RSILongStrategy(Strategy):
    rsi_periods = 7
    max_periods = 9
    def init(self):
        self.rsi = self.I(lambda x: x, self.data.rsi)

    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) -1
        for trade in self.trades:
            if current_bar - trade.entry_bar >= periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)
    def next(self):
        self.CloseOldPosition(self.max_periods)
        if len(self.rsi) < 3:
            return
        rsi_left = self.rsi[-5:-2]
        rsi_edge = self.rsi[-2]
        rsi_right = self.rsi[-1]
        # print("rsi: ", self.rsi[-10:])

        if rsi_right >= 75:
            if rsi_edge > rsi_right and rsi_edge >= max(rsi_left):
                self.position.close()
                # self.sell(sl=self.data.Close[-1] * 1.02, tp=self.data.Close[-1] * 0.95)  # Example SL/TP
        elif rsi_right <= 25:
            if rsi_edge < rsi_right and rsi_edge <= min(rsi_left):
                self.position.close()
                self.buy(sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP

class RSI7LongStrategy_9p(RSILongStrategy):
    rsi_periods = 7
    max_periods = 9

class RSI7LongStrategy_26p(RSILongStrategy):
    rsi_periods = 7
    max_periods = 26
# stras = RSI7LongStrategy_9p(data = df)
# stras.data

## ADX long Pattern

In [ ]:
class ADXLongStrategy(Strategy):
    max_periods = 9
    def init(self):
        self.adx = self.I(lambda x: x, self.data.adx)

        predict_trend = []
        for i in range(len(self.data.index)):
            if i < 5:
                predict_trend.append(0)
                continue

            if min(self.data.adx[i-4:i+1]) == self.data.adx[i-1] or max(self.data.adx[i-4:i+1]) == self.data.adx[i-1]:  # edge point
                low_trend = self.data.Low[i-5:i-1] < self.data.Low[i-1]
                high_trend = self.data.High[i-5:i-1] < self.data.High[i-1]
                close_trend = self.data.Close[i-5:i-1] < self.data.Close[i-1]
                if (low_trend+high_trend+close_trend) >= 2:  # current trend up 
                    predict_trend.append(-1)                
                else:  # current trend down 
                    predict_trend.append(1)
            else:
                predict_trend.append(0)

        self.predict_trend = self.I(lambda x: x, predict_trend)


    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) - 1
        for trade in self.trades:
            if current_bar - trade.entry_bar >= periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)
    def next(self):
        self.CloseOldPosition(self.max_periods)
        # if len(self.predict_trend) >18:
            # print(self.data.Close)
            # print(self.adx)
            # print(self.predict_trend)
        if self.predict_trend == 1:
            self.position.close()
            self.buy(sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP
        elif self.predict_trend == -1:
            self.position.close()
            # self.sell(sl=self.data.Close[-1] * 1.02, tp=self.data.Close[-1] * 0.95)  # Example SL/TP

class ADXLongStrategy_9p(ADXLongStrategy):
    max_periods = 9

class ADXLongStrategy_26p(ADXLongStrategy):
    max_periods = 26
# stras = RSI7LongStrategy_9p(data = df)
# stras.data

In [106]:
# df
# bt = FractionalBacktest(df[-20:], ADXLongStrategy_9p, cash=10_000, commission=0.002)
# stats = bt.run()
# bt.plot()


## Bullish Pin Bar

In [139]:
def SMA(values, n: int) -> np.ndarray:
    """
    Return simple moving average of `values`, at each step taking into account `n` previous values.
    This returns a numpy array of the same length as values.
    """
    # Make it a pandas Series:
    s = pd.Series(values)
    sma = s.rolling(n, min_periods=1).mean()
    return sma.values  # array with same length

class BullishPinBarStrategy(Strategy):
    max_periods = 9
    wick_body_ratio = 2.0       # wick at least this × body
    min_wick_pct = 0.6          # wick at least this % of candle height
    min_range_atr = 0.8         # candle total range ≥ this × ATR
    trend_sma_period = 50       # for trend filter

    def init(self):

        # self.sma_trend = pd.Series(self.data.Close).rolling(window=self.trend_sma_period, min_periods=1).mean()
        # self.sma_trend = self.I(pd.Series.rolling, pd.Series(self.data.Close), self.trend_sma_period).mean()
        self.sma_trend = self.I(SMA, self.data.Close.to_series(), self.trend_sma_period)

    def CloseOldPosition(self, periods=9):
        current_bar = len(self.data.index) - 1
        for trade in self.trades:
            if current_bar - trade.entry_bar >= periods:
                trade.close()
                # print("Closed old position at bar", current_bar, trade.entry_bar, " for trade", trade)
    def next(self):
        body = abs(self.data.Close[-1] - self.data.Open[-1])
        upper_wick = self.data.High[-1] - max(self.data.Close[-1], self.data.Open[-1])
        lower_wick = min(self.data.Close[-1], self.data.Open[-1]) - self.data.Low[-1]
        
        # require enough ATR range
        if self.data.tr[-1] < (self.min_range_atr * self.data.atr14[-1]):
            return  # skip, candle too small
        
        # require small body relative to range
        if body > ( (1 - self.min_wick_pct) * self.data.tr[-1] ):
            return
        
        # trend filter: define uptrend / downtrend
        # For example, price above SMA → uptrend, below → downtrend
        # print(self.data.Close, self.sma_trend)
        uptrend = self.data.Close[-1] > self.sma_trend[-1]
        downtrend = self.data.Close[-1] < self.sma_trend[-1]

        # bullish pin bar
        if (lower_wick >= self.wick_body_ratio * body) and uptrend:  # and (self.data.Close[-1] > self.data.Open[-1]):
            # entry: long
            # set stop-loss and take-profit
            self.position.close()
            self.buy(sl=self.data.Close[-1] * 0.98, tp=self.data.Close[-1] * 1.05)  # Example SL/TP
            # sl = self.data.Low[-1]  # below the low of pin bar
            # tp = self.data.Close[-1] + (self.data.Close[-1] - sl) * 2  # risk:reward = 1:2
            # self.buy(sl = sl, tp = tp)

        elif upper_wick >= self.wick_body_ratio * body and downtrend:   # and self.data.Open[-1] > self.data.Close[-1]
            self.position.close()
            # sl = self.data.High[-1]
            # tp = self.data.Close[-1] - (sl - self.data.Close[-1]) * 2
            # self.sell(sl=sl, tp=tp)


class BullishPinBarStrategy_9p(BullishPinBarStrategy):
    max_periods = 9

class BullishPinBarStrategy_26p(BullishPinBarStrategy):
    max_periods = 26
# stras = RSI7LongStrategy_9p(data = df)
# stras.data

In [140]:
df
bt = FractionalBacktest(df[-100:], BullishPinBarStrategy_9p, cash=10_000, commission=0.002)
stats = bt.run()
bt.plot()


C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\backtesting\_util.py:135: UserWarning: `.to_series()` is deprecated. For pd.Series conversion, use accessor `.s`
  warnings.warn("`.to_series()` is deprecated. For pd.Series conversion, use accessor `.s`")


FractionalBacktest.run:   0%|          | 0/99 [00:00<?, ?bar/s]

C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\bokeh\util\serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p11371', ...)

# All in one

In [95]:
def backtest_function(symbol, timeframe, strategies, cash=10_000, commission=0.002):
    symbol = symbol.upper()
    timeframe = timeframe.lower()
    tag = f"{symbol}_{timeframe}"
    df = get_data_4_8(symbol, timeframe, ['rsi7', 'adx'])
    try:
        print("df.shape: ", df.shape)
    except Exception as e:
        print("df: ", df)
        raise e
    fractional_unit = pow(10, 2-int(np.log10(df['High'].max())))
    # _stats, _plot = [], []
    for strategy in strategies:
        max_periods = strategy.max_periods
        print(f"Running backtest for {strategy.__name__} with max_periods={max_periods}")
        bt = FractionalBacktest(df, strategy, cash=cash, commission=commission, fractional_unit=fractional_unit)
        stats = bt.run()
        save_stats(stats, bt, "backtest_summary.csv", tag)

In [139]:
a = backtest_function("ETHUSDT", "4h", [TriplePatternStrategy_9p, TriplePatternStrategy_26p, ButterflyStrategy_9p, ButterflyStrategy_26p])

Running backtest for TriplePatternStrategy_9p with max_periods=9
Running backtest for TriplePatternStrategy_26p with max_periods=26
Running backtest for ButterflyStrategy_9p with max_periods=9
Running backtest for ButterflyStrategy_26p with max_periods=26


In [ ]:
symbols = ['ETHUSDT', 'HBARUSDT']  # , 'ETHUSDT'
tfs = ["p1h"]
for symbol in symbols:
    for tf in tfs:
        a = backtest_function(symbol, tf, [TriplePatternStrategy_9p, TriplePatternStrategy_26p, ButterflyStrategy_9p, ButterflyStrategy_26p])

In [ ]:
symbols = ['ETHUSDT', 'HBARUSDT']  # , 'ETHUSDT'
tfs = ["f1h"]
for symbol in symbols:
    for tf in tfs:
        a = backtest_function(symbol, tf, [ADXLongStrategy_9p, ADXLongStrategy_26p, RSI7LongStrategy_9p, RSI7LongStrategy_26p])

In [141]:
df_1d = get_data_4_8("ETHUSDT", "1d")
bt = FractionalBacktest(df_1d, TriplePatternStrategy_9p, cash=10_000, commission=0.002, fractional_unit=0.01)
df_1d_stat = bt.run()

,Open,Close,High,Low,Volume
open_time,,,,,
2025-05-13 00:00:00+00:00,0.20554,0.21612,0.21857,0.20419,183175906.0
2025-08-28 00:00:00+00:00,0.23762,0.23922,0.24201,0.23540,134719322.0
2025-08-29 00:00:00+00:00,0.23922,0.22796,0.24037,0.22349,249640545.0
2025-08-30 00:00:00+00:00,0.22797,0.22574,0.22951,0.22028,107200716.0
2025-08-31 00:00:00+00:00,0.22575,0.21896,0.22871,0.21784,79075844.0


In [64]:
df[-20:]

,Open,Close,High,Low,Volume,rsi,rsi14,adx
open_time,,,,,,,,
2025-08-30 21:00:00+00:00,0.22534,0.22494,0.22556,0.22470,1501365.0,47.303222,45.559180,4.395721
2025-08-30 22:00:00+00:00,0.22494,0.22618,0.22664,0.22468,2316734.0,57.481326,50.011356,4.316264
2025-08-30 23:00:00+00:00,0.22619,0.22574,0.22680,0.22528,3203916.0,53.225532,48.495815,4.363417
2025-08-31 00:00:00+00:00,0.22575,0.22778,0.22809,0.22561,3029463.0,66.601057,55.264625,4.445924
2025-08-31 01:00:00+00:00,0.22779,0.22832,0.22871,0.22664,2608834.0,69.311200,56.880084,4.712661
2025-08-31 02:00:00+00:00,0.22834,0.22765,0.22857,0.22746,1356011.0,62.025705,54.261869,5.098667
2025-08-31 03:00:00+00:00,0.22766,0.22774,0.22822,0.22706,1738604.0,62.641114,54.564416,5.349958
2025-08-31 04:00:00+00:00,0.22775,0.22693,0.22784,0.22648,1991090.0,53.531983,51.276927,5.462870
2025-08-31 05:00:00+00:00,0.22692,0.22591,0.22692,0.22559,2057496.0,44.108673,47.403749,5.312094


In [63]:
int(datetime(2025, 9, 1, 0, 0, 0).timestamp())

1756659600